In [1]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np

pd.set_option("display.max_rows", 500)
pd.set_option("display.max_columns", 500)
pd.set_option("display.width", 1000)
pd.set_option("display.max_columns", None)
pd.set_option("max_colwidth", 800)
pd.set_option("display.max_colwidth", None)

import warnings

warnings.simplefilter(action="ignore", category=FutureWarning)

In [3]:
artists = ["Sanah"]

artists = [x.replace(" ", "_").lower() for x in artists]
artists

['sanah']

In [7]:
def song_scraper(artists: list):
    dfs = []
    df_text = pd.DataFrame(data={"Original": [], "Translated": []})

    part_link = "https://www.tekstowo.pl"

    for i in artists:
        artist_link = (
            f"https://www.tekstowo.pl/piosenki_artysty,{i},alfabetycznie,strona,1.html"
        )

        page = requests.get(artist_link)
        soup = BeautifulSoup(page.text, "lxml")

        max_num_page = soup.find_all("a", class_="page-link")[-2].text
        max_num_page = int(max_num_page)

        for num_page in range(1, max_num_page + 1):
            boxes = soup.find_all("div", class_="flex-group")
            data = {"Link": []}

            for box in boxes:
                try:
                    link = box.find("a", class_="title").get("href")
                    song_link = part_link + link

                    data["Link"].append(song_link)
                except:
                    pass

            df = pd.DataFrame(data)
            dfs.append(df)

            next_number_page = num_page + 1
            next_page = f"https://www.tekstowo.pl/piosenki_artysty,{i},alfabetycznie,strona,{next_number_page}.html"

            page = requests.get(next_page)
            soup = BeautifulSoup(page.text, "lxml")

    df_songs = pd.concat(dfs, ignore_index=True)

    for _, row in df_songs.iterrows():
        try:
            song_page = requests.get(row[0])

            soup = BeautifulSoup(song_page.text, "lxml")

            original = soup.find_all("div", class_="inner-text")[0].text
            translated = soup.find_all("div", class_="inner-text")[1].text
            translated = np.nan if translated == "" else translated

            if re.search(r"[ąćęłńóśźżĄĆĘŁŃÓŚŹŻ]", original):
                df_text = pd.concat(
                    [df_text, pd.DataFrame({"Original": [original], "Translated": [translated]})],
                    ignore_index=True
                )

            else:
                df_text = pd.concat(
                    [df_text, pd.DataFrame({"Original": [translated], "Translated": [original]})],
                    ignore_index=True
                )
        except:
            pass 

    return df_text


df = song_scraper(artists)

In [10]:
df.head()

,Link
0,"https://www.tekstowo.pl/piosenka,sanah,_ii__da_bog_kiedys_zasiasc_w_polsce_wolnej__a__asnyk_.html"
1,"https://www.tekstowo.pl/piosenka,sanah,_i__da_bog_kiedys_zasiasc_w_polsce_wolnej__a__asnyk_.html"
2,"https://www.tekstowo.pl/piosenka,sanah,_hot16challenge2.html"
3,"https://www.tekstowo.pl/piosenka,sanah,2_00.html"
4,"https://www.tekstowo.pl/piosenka,sanah,2_10.html"


In [9]:
def text_scrap(df):
    df_text = None
    df_text = pd.DataFrame(data={"Polish": [], "English": []})

    for index, row in df.iterrows():
        try:
            page = requests.get(row[0])
            # print(page)
            soup = BeautifulSoup(page.text, "lxml")

            original = soup.find_all("div", class_="inner-text")[0].text
            translated = soup.find_all("div", class_="inner-text")[1].text

            translated = np.nan if translated == "" else translated

            if re.search(r"[ąćęłńóśźżĄĆĘŁŃÓŚŹŻ]", original):
                df_text = df_text.append(
                    {"Polish": original, "English": translated}, ignore_index=True
                )

            else:
                df_text = df_text.append(
                    {"Polish": translated, "English": original}, ignore_index=True
                )
        except:
            pass

    return df_text


df_text = text_scrap(df)

In [12]:
print(len(df_text))
print(len(df))
df_text.iloc[[1896]]

3276
3294


,Polish,English
1896,"Stoję na moście\r\nCzekam w ciemnościach \r\nMyślałam, że tutaj będziesz\r\nPo za deszczem nie ma nic\r\nŻadnych śladów stóp na ziemi\r\nSłucham, ale nie ma tu żadnego dźwięku\n\r\nCzy nikt nie próbuje mnie odnaleźć?\r\nCzy nikt nie zabierze mnie do domu?\n\r\nJest cholernie zimna noc,\r\nPróbuję rozgryźć to życie\r\nCzy nie weźmiesz mnie za rękę?\r\nZabierz mnie w jakieś inne miejsce\r\nNie wiem kim jesteś\r\nAle ja...Jestem z Tobą\r\nJestem z Tobą\n\r\nSzukam miejsca\r\nSzukam twarzy\r\nCzy jest tu ktoś, kogo znam\r\nPonieważ nic nie idzie zgodnie z planem\r\nI wszystko jest niepoukładane\r\nI nikt nie chce być sam\n\r\nCzy nikt nie próbuje mnie odnaleźć?\r\nCzy nikt nie zabierze mnie do domu?\n\r\nJest cholernie zimna noc\r\nPróbuję rozgryźć to życie\r\nCzy nie weźmiesz mnie za rękę?\r\nZabierz mnie w jakieś inne miejsce\r\nNie wiem kim jesteś\r\nAle ja...Jestem z Tobą\n\r\nDlaczego wszystko jest takie mylące\r\nMoże po prostu odchodzę od zmysłów\r\nyeah yeah yeah\n\r\nJest cholernie zimna noc\r\nPróbuję rozgryźć to życie\r\nCzy nie weźmiesz mnie za rękę?\r\nZabierz mnie w jakieś inne miejsce\r\nNie wiem kim jesteś\r\nAle ja...Jestem z Tobą\r\nJestem z Tobą\n\r\nWeź mnie za rękę\r\nZabierz mnie w jakieś inne miejsce\r\nNie wiem kim jesteś\r\nAle ja...Jestem z Tobą\r\nJestem z Tobą\n\r\nWeź mnie za rękę\r\nZabierz mnie w jakieś inne miejsce\r\nNie wiem kim jesteś\r\nAle ja...Jestem z Tobą\r\nJestem z Tobą\r\nJestem z Tobą\n\n\r\nTłumaczenie dodał(a): she's_artemis","I'm standing on the bridge\r\nI'm waiting in the dark\r\nI thought that you'd be here by now\r\nThere's nothing but the rain\r\nNo footsteps on the ground\r\nI'm listening, but there's no sound\n\r\nIsn't anyone trying to find me?\r\nWon't somebody come take me home?\n\r\nIt's a damn cold night\r\nTrying to figure out this life\r\nWon't you take me by the hand\r\nTake me somewhere new?\r\nDon't know where you are, but I\r\nI'm with you\r\nI'm with you\n\r\nI'm looking for a place\r\nI'm searching for a face\r\nIs anybody here I know?\r\n'Cause nothing's going right\r\nAnd everything's a mess\r\nAnd no one likes to be alone\n\r\nIsn't anyone trying to find me?\r\nWon't somebody come take me home?\n\r\nIt's a damn cold night\r\nTrying to figure out this life\r\nWon't you take me by the hand\r\nTake me somewhere new?\r\nDon't know where you are, but I\r\nI'm with you\r\nI'm with you\n\r\nOh, why is everything so confusing?\r\nMaybe I'm just out of my mind\r\nYeah-he-yaa, yeah-he-yah,\r\nyeah-he-yah, yeah-he-yah, yeah!\n\r\nIt's a damn cold night\r\nTrying to figure out this life\r\nWon't you take me by the hand\r\nTake me somewhere new?\r\nDon't know who you are, but I\r\nI'm with you\r\nI'm with you\n\r\nTake me\r\nTake me by the hand\r\nTake me somewhere new\r\nDon't know who you are, but I\r\nI'm with you\r\nI'm with you\n\r\nTake me by the hand\r\nTake me somewhere new\r\nDon't know who you are, but I\r\nI'm with you\r\nI'm with you\r\nI'm with you"


In [13]:
import pickle

with open("dataframe_text", "wb") as f:
    pickle.dump(df_text, f)

In [14]:
df_text["English"].isna().sum()

316

In [9]:
len(df_text)

1864

In [13]:
df.iloc[[799]]

,Link
799,"https://www.tekstowo.pl/piosenka,drake,cameras__official_remix_.html"


In [11]:
# 400 wiersz - polski i angielski razem (dla pierwszych dwoch z listy new)
df_text.iloc[[1509]]

,Polish,English
1509,"[Ariana Grande]\nDokąd się zwrócić?\nGdy nie ma do kogo?\nDokąd, gdy tak ciężko jest?\nJak dokonać pierwszych zmian?\nJak się uczyć?\nCzego ci trzeba?\n\n[Pan Bagietka]\nNadziei\n\n[Ariana Grande]\nNadzieja jest pieśnią\nGdy świat ci zagraża\nNadzieja to otwarte drzwi\n\n[Ariana Grande i Jim Carrey]\nA może upadki są po to, by...\nPodnosić się z nich?",[Ariana Grande]\nWhere do you turn?\nWhen there’s no one to turn to?\nWhen it's so hard to cope\nHow can you start to make a change?\nMaybe learn to\nWhat do you need?\n\n[Mr. Baggett]\nHope\n\n[Ariana Grande]\nHope is the song\nWhen the world is a menace\nHope is an open door\n\n[Ariana Grande & Jim Carrey]\nMaybe getting back up again is\nWhat falls are for


In [12]:
df_text.iloc[[799]]

,Polish,English
799,NaN,"[Intro]\nHow you mean, how you mean\nI Thought you knew about the team, (ay)\nHow you mean, how you mean\nI Thought you knew about the team, (ay)\n\n[Verse 1: Ke'Ondris]\nBefore they knew who i was tha preety girls had a problem\nWith me but tha ugly girls always wanted to holla\nBut i couldn't settle for less so i threw tha\nLight skinned girls a couple dollas\nNow she my bestfrind and my brand new girlfriend\nShe only wanna talk 2 me\nBut only on camera\nOnly on camera\nOnly on camera\nShe say they in love with me\nOnly on camera\nOnly on camera\nOnly on camera\nIm Stuck without ah move im\nIm lost in tha groove I\nGotta figure out something to remove ya\nFrom my life cus you fucking my shyt up\nAnd now you living tha single life\nAnd you need my help\n\n[Verse 2: Peter Ray Rivers]\nFrom tha botton of my heart I mean every word that i say\nSay you dont care\nSay you really want it\nI want it\nIm stuttering\n""I"" ""I"" ""I""\nIm on my grind like cc said\nCus this line hot\nThey ah ride around town\nLooking for some dumb muphuka's\n2 go f*ck up\nYou think you winning\nBut chu ain't winning\nYeah & that how ya story goes\n\n[Bridge]\nOoh finally got you right here\nTonight I'll ease your mind\nThat's why I'm calling on you\nAnd ooh soft your loves desire\nIt's hard to stay away\nYou keep me calling on you\n\n[Hook]\nHow you mean, how you mean\nThought you knew about the team, (ay)\nHow you mean, how you mean\nThought you knew about the team, (ay)\n\n[Verse 3: Drake]\nHow you mean, how you mean, what you know about the team\nYou just know what you get told girl I see behind the scene\nShe spilled whiskey on her shirt, she gonna have to get it cleaned\nShe been going way to hard, someone has to intervene\nBaby she look like a star, but only on camera\nOnly on camera, only on camera\nIt might look like I care, but only on camera\nOnly on camera, only on camera\nDon't listen to the lies, I swear they all lies\nYou know I could be your knight in shining armor all tires\nGirl, they love me like I'm Prince, like the new kid with the crown\nBunch of underground kings, thought you knew how we get down (ah)\n\n[Bridge]\n\n[Hook]"


In [5]:
df.duplicated().sum()

0

In [6]:
df[df["Link"] == "https://www.tekstowo.pl/piosenka,taylor_swift,dorothea.html"]

,Link
79,"https://www.tekstowo.pl/piosenka,taylor_swift,dorothea.html"
